In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy

In [20]:
df = pd.read_pickle('../sparse_ratings.pkl')

In [21]:
df.head(10)

userId,1,2,3,4,5,6,7,8,9,10,...,330966,330967,330968,330969,330970,330971,330972,330973,330974,330975
movieId,,,,,,,,,,,,,,,,,,,,,
1203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
527,0.0,5.0,0.0,0.0,4.0,0.0,4.0,0.0,4.0,0.0,...,0.0,4.0,0.0,0.0,0.0,4.5,4.0,0.0,4.5,3.5
1193,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,4.0,3.0
750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,...,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26082,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1178,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6818,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
926,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
160718,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
movies_info = pd.read_csv("../suitable_movies.csv")

In [23]:
movies_info.set_index("movieId", inplace=True)

In [24]:
def filter_users(df=df, n=1, categories=4):
    movie_genres = movies_info['suitable_genre']
    df_ = df.copy()
    valid_users = []
    for user in df_.columns:
        genres = {
            "Action": 0,
            "Adventure": 0,
            "Animation": 0,
            "Comedy": 0,
            "Crime": 0,
            "Drama": 0,
            "Fantasy": 0,
            "Horror": 0,
            "Mystery": 0,
            "Romance": 0,
            "Sci-Fi": 0,
            "Thriller": 0
        }
        user_data = df_[df_[user] != 0.0][user]
        movie_genres = movies_info[movies_info.index.isin(user_data.index)]['suitable_genre']
        for movie_genre in movie_genres:
            genres[movie_genre] += 1
        #print(genres)
        one_more_than_n = False
        valid_user_counter = 0
        for genre, count in genres.items():
            if count >= n:
                valid_user_counter += 1
            if count > n: one_more_than_n = True
        # if the user has at least n movies in each category, and they have at least one movie to be predicted - we say they are valid for our purposes, and we store their id (column name of the df_)
        if valid_user_counter >= categories and one_more_than_n:
            valid_users.append(user)
            #print(f'found a valid user - {user}')
    return valid_users

In [25]:
from sklearn.utils import shuffle


def filter_ratings(valid_users, df=df, n=2, categories=4):
    filtered = []
    for user_id in valid_users:
        genres = {
            "Action": 0,
            "Adventure": 0,
            "Animation": 0,
            "Comedy": 0,
            "Crime": 0,
            "Drama": 0,
            "Fantasy": 0,
            "Horror": 0,
            "Mystery": 0,
            "Romance": 0,
            "Sci-Fi": 0,
            "Thriller": 0
        }
        user_ratings = df[user_id][df[user_id] != 0.0]
        user_genres = movies_info[movies_info.index.isin(user_ratings.index)]['suitable_genre']
        #looks like double work but here we are ensuring only the genres that were rated at least n times are used
        for movie_genre in user_genres:
            genres[movie_genre] += 1
        unique_user_genres = [key for key, value in genres.items() if value >= n]
        # then making sure that we are picking the genres randomly (we will cut it later)
        user_genres = shuffle(user_genres)
        # for every unique genre the user has rated - pick n random movies
        lst_filtered_movies = []
        lst_filtered_out_movies = []
        # making sure every genre is represented the same amount
        for genre in unique_user_genres[:categories]:
            #print(genre)
            movies_of_genre = user_genres[user_genres == genre].index.values
            df_movies_of_genre = shuffle(pd.DataFrame(data=movies_of_genre))
            #print(df_movies_of_genre[:n])
            lst_filtered_movies.append(df_movies_of_genre[:n])
            
        lst_filtered_movies = pd.concat(lst_filtered_movies).values
        for movie_id in user_ratings.index.values:
            if movie_id not in lst_filtered_movies:
                lst_filtered_out_movies.append(movie_id)
        filtered.append([user_id, [x[0] for x in lst_filtered_movies], lst_filtered_out_movies])
    return_df = pd.DataFrame(data=filtered, columns=['userId', 'testMovies', 'evalMovies'])

    return return_df.set_index('userId')

In [26]:
# bootstrapping with the first resulting df - should be way faster - it took about 6 hours to make - now we are going to help ourselves with it since any other dataframe can only be smaller and it is way faster to compare 20k user than 300k users
import pandas as pd
df_boot = pd.read_pickle('n1_c7_full')
####################

In [27]:
indices_1_7 = [x for x in df_boot.index.values]
df = df[indices_1_7]

In [28]:
df

userId,41,50,82,94,95,105,123,137,144,148,...,330905,330908,330914,330917,330921,330922,330934,330938,330941,330949
movieId,,,,,,,,,,,,,,,,,,,,,
1203,0.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.5,5.0,4.0,0.0,0.0
527,5.0,0.0,2.0,4.0,0.0,5.0,5.0,4.5,0.0,1.0,...,5.0,4.0,4.5,2.0,0.0,5.0,0.0,5.0,0.0,4.0
1193,0.0,0.0,0.0,0.0,0.0,3.0,5.0,3.5,3.0,1.0,...,0.0,0.0,4.5,3.5,3.5,0.0,5.0,5.0,0.0,4.0
750,0.0,4.5,0.0,4.0,0.0,5.0,5.0,4.0,0.0,3.0,...,5.0,0.0,0.0,4.5,0.0,0.0,5.0,0.0,0.0,4.0
26082,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1981,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1760,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
6482,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
import time

all_ns = [i for i in range(1, 5)]
all_cats = [i for i in range(7, 12)]
for n in all_ns:
    beginning = time.time()
    print(beginning)
    for cat in all_cats:
        df_loop = df#shuffle(df.iloc[:,:sample_size].T).T
        valid_users = filter_users(df=df_loop, n=n, categories=cat)
        print('found all valid users')
        eval_df = filter_ratings(valid_users=valid_users, df=df_loop, n=n, categories=cat)
        eval_df.to_pickle(f'n{n}_c{cat}_full')
        print(f'n{n}_c{cat}: {eval_df}')
        print(f'time: {time.time() - beginning}')

1714593840.7104187
found all valid users
n1_c7:                                                testMovies  \
userId                                                      
41                    [181, 158, 616, 135, 448, 177, 457]   
50      [79553, 93831, 110882, 168250, 913, 109487, 14...   
82                  [9, 1017, 3483, 1503, 34, 1983, 3354]   
94                [2951, 4327, 750, 2726, 346, 1258, 457]   
95       [1304, 2096, 86833, 90057, 179085, 1035, 164179]   
...                                                   ...   
330922  [114662, 140110, 166946, 162600, 168250, 10948...   
330934               [1304, 609, 144, 261, 913, 900, 457]   
330938         [3444, 1031, 2137, 2060, 2312, 2017, 2789]   
330941          [986, 4090, 3146, 2908, 2017, 1105, 4814]   
330949     [7451, 2726, 1193, 25771, 121231, 109487, 457]   

                                               evalMovies  
userId                                                     
41      [527, 475, 529, 508, 73, 535, 